In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
from bs4 import BeautifulSoup
import requests
import glob

In [ ]:
# # 파일 경로 패턴 설정 (여기서는 forecast_*.csv 파일만 가져옴)
# file_list = glob.glob('../data/raw/forecast_*.csv')

# # 모든 파일을 읽어서 하나의 데이터프레임으로 합치기
# df_list = [pd.read_csv(file, encoding='euc-kr') for file in file_list]
# forecast_df = pd.concat(df_list, ignore_index=True)

# # CSV 파일로 저장
# forecast_df.to_csv('../data/processed/forecast_merged2.csv', index=False)


In [ ]:
# 파일 경로 설정
file_path = '../data/processed/forecast_merged2.csv'
forecast_df = pd.read_csv(file_path)


#일시 데이트타임 적용
forecast_df['일시'] = pd.to_datetime(forecast_df['일시'])
forecast_df['연도'] = forecast_df['일시'].dt.year
forecast_df['월'] = forecast_df['일시'].dt.month
forecast_df['일'] = forecast_df['일시'].dt.day
forecast_df['시간'] = forecast_df['일시'].dt.hour
forecast_df['요일'] = forecast_df['일시'].dt.day_name().str[:3]

# 결측치 제거
forecast_df.fillna(0, inplace=True)

#데이터 타입 숫자인지 확인????
numeric_columns = forecast_df.select_dtypes(include=['float64', 'int64']).columns
forecast_df[numeric_columns] = forecast_df[numeric_columns].apply(pd.to_numeric, errors='coerce')


# 결과 확인
forecast_df.head(-5)

In [ ]:
# 실시간 추가(셀레니움?) -- 강사님께 제지당함

# url = 'https://data.kma.go.kr/data/sea/selectBuoyRltmList.do?pgmNo=52'

# # 2️⃣ HTTP 요청을 보내서 웹 페이지 가져오기
# response = requests.get(url)

# # 3️⃣ BeautifulSoup을 사용해 HTML 파싱
# soup = BeautifulSoup(response.text, 'html.parser')

# # 4️⃣ 원하는 데이터 추출 (예: 모든 제목 태그 <h1>)
# titles = soup.find_all('h1')

# # 5️⃣ 추출한 데이터를 출력
# for title in titles:
#     print(title.get_text())

In [ ]:
# 1️⃣ 지점 22105 데이터 필터링 (2020년 1월 1일부터 2025년 1월 12일까지)
filtered_df = forecast_df[(forecast_df['지점'] == 22105) & (forecast_df['일시'] >= '2020-01-01') & (forecast_df['일시'] <= '2025-01-12')]

# 2️⃣ '일시'를 인덱스로 설정
filtered_df.set_index('일시', inplace=True)

# 3️⃣ 기온 변화 시각화
plt.figure(figsize=(15, 6))
plt.plot(filtered_df.index, filtered_df['기온(°C)'], color='orange', linestyle='-', marker='o', markersize=2)
plt.title('지점 22105 - 2020년 1월 1일부터 2025년 1월 12일까지 기온 변화')
plt.xlabel('일자')
plt.ylabel('기온(°C)')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

# 4️⃣ 수온 변화 시각화
plt.figure(figsize=(15, 6))
plt.plot(filtered_df.index, filtered_df['수온(°C)'], color='blue', linestyle='-', marker='o', markersize=2)
plt.title('지점 22105 - 2020년 1월 1일부터 2025년 1월 12일까지 수온 변화')
plt.xlabel('일자')
plt.ylabel('수온(°C)')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


plt.figure(figsize=(12, 6))
yearly_temp = forecast_df.groupby('연도')['수온(°C)'].mean()
plt.plot(yearly_temp.index, yearly_temp.values, marker='o', color='blue')
plt.title('연도별 평균 수온 변화')
plt.xlabel('연도')
plt.ylabel('수온(°C)')
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
hourly_wave = forecast_df.groupby('연도')['평균파고(m)'].mean()
plt.plot(hourly_wave.index, hourly_wave.values, marker='o', color='green')
plt.title('연도별 평균파고 변화')
plt.xlabel('연도')
plt.ylabel('평균균파고(m)')
plt.grid(True)
plt.show()

In [ ]:
Z